In [36]:
import pandas as pd
import numpy as np
import time
import os
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pickle
import re

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

chrome_options = Options()
chrome_options.add_argument('--headless')  # 웹 브라우저를 표시하지 않고 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [60]:
total_problem_number_list = [i for i in range(1000, 31285)]
total_problem_name_list = []
total_problem_tier_list = []
total_problem_algorithm_list = []
total_crawling_problem_number_list = []
total_problem_time_condition_list = []
total_problem_memory_condition_list = []
total_problem_submission_count_list = []
total_problem_answer_submission_count_list = []
total_problem_answered_people_count_list = []
total_problem_correct_rate_list = []

total_problem_description_list = []
total_problem_input_list = []
total_problem_output_list = []

remove_problem_list = []
total_english_problem_list = []

In [59]:
tier_dict = {
    'Not ratable':0,
    'Bronze V': 1,
    'Bronze IV': 2,
    'Bronze III': 3,
    'Bronze II': 4,
    'Bronze I': 5,
    'Silver V': 6,
    'Silver IV': 7,
    'Silver III': 8,
    'Silver II': 9,
    'Silver I': 10,
    'Gold V': 11,
    'Gold IV': 12,
    'Gold III': 13,
    'Gold II': 14,
    'Gold I': 15,
    'Platinum V': 16,
    'Platinum IV': 17,
    'Platinum III': 18,
    'Platinum II': 19,
    'Platinum I': 20,
    'Diamond V': 21,
    'Diamond IV': 22,
    'Diamond III': 23,
    'Diamond II': 24,
    'Diamond I': 25,
    'Ruby V': 26,
    'Ruby IV': 27,
    'Ruby III': 28,
    'Ruby II': 29,
    'Ruby I': 30,
    'Master':31
}
flipped_tier_dict = {
    0: 'Not ratable',
    1: 'Bronze V',
    2: 'Bronze IV',
    3: 'Bronze III',
    4: 'Bronze II',
    5: 'Bronze I',
    6: 'Silver V',
    7: 'Silver IV',
    8: 'Silver III',
    9: 'Silver II',
    10: 'Silver I',
    11: 'Gold V',
    12: 'Gold IV',
    13: 'Gold III',
    14: 'Gold II',
    15: 'Gold I',
    16: 'Platinum V',
    17: 'Platinum IV',
    18: 'Platinum III',
    19: 'Platinum II',
    20: 'Platinum I',
    21: 'Diamond V',
    22: 'Diamond IV',
    23: 'Diamond III',
    24: 'Diamond II',
    25: 'Diamond I',
    26: 'Ruby V',
    27: 'Ruby IV',
    28: 'Ruby III',
    29: 'Ruby II',
    30: 'Ruby I',
    31: 'Master'
 }


In [58]:
driver = webdriver.Chrome('chromedriver.exe')
#driver = webdriver.Chrome('chromedriver.exe', options=chrome_options)
driver.get("https://www.acmicpc.net/login?next=%2Fproblem%2F1000")

username_input = driver.find_element_by_name("login_user_id")  # 사용자명 필드의 name 속성값에 맞게 수정
password_input = driver.find_element_by_name("login_password")  # 비밀번호 필드의 name 속성값에 맞게 수정
login_button = driver.find_element_by_xpath("//*[@id='submit_button']")  # 로그인 버튼의 XPath에 맞게 수정

username_input.send_keys("llsy159")
password_input.send_keys("llsy13579!")
login_button.click()

In [78]:

for n in total_problem_number_list:

    if((n in total_crawling_problem_number_list) or (n in total_english_problem_list) or (n in remove_problem_list)):
        continue

    problem_url = 'https://www.acmicpc.net/problem/' + str(n)
    print(problem_url)
   
    driver.get(problem_url)

    try:
        problem_description = driver.find_element_by_css_selector('#problem_description')
    except NoSuchElementException:
        remove_problem_list.append(n)
        print('존재하지 않는 문제')
        continue
    
    if(not any(char >= '가' and char <= '힣' for char in problem_description.text)):

        total_english_problem_list.append(n)
        print('영어 문제')
        continue

    problem_info_object = driver.find_element_by_css_selector('#problem-info')
    problem_info_list = [float(pio) if '.' in pio else int(pio) for pio in re.findall(r'\d+\.\d+|\d+', problem_info_object.text.split('\n')[1])]
    
    if(len(problem_info_list)!=6):
        
        remove_problem_list.append(n)
        print('정상적이지 않은 문제')
        continue
    
    problem_name_object = driver.find_element_by_css_selector('#problem_title')
    problem_tier_object = driver.find_element_by_css_selector('.solvedac-tier')


    
    total_problem_name_list.append(problem_name_object.text)
    try:
        total_problem_tier_list.append(flipped_tier_dict[int(re.search(r'/tier/(\d+)\.svg', problem_tier_object.get_attribute('src')).group(1))])
    except AttributeError:
        total_problem_tier_list.append('Zero')
    
    
    try:
        problem_algorithm_object = driver.find_element_by_css_selector('.show-spoiler')
        algorithm_button = driver.find_element_by_xpath("//*[@id='problem_tags']/div[2]/p/a")
        algorithm_button.click()
        problem_algorithm_object = driver.find_element_by_css_selector('.spoiler-list')
        total_problem_algorithm_list.append(problem_algorithm_object.text.split('\n'))
    except NoSuchElementException: 
        try:
            problem_algorithm_object = driver.find_element_by_css_selector('.spoiler-list')
            total_problem_algorithm_list.append(problem_algorithm_object.text.split('\n'))
        except NoSuchElementException: 
            total_problem_algorithm_list.append('Not Exist Algorithm')

    
    total_problem_time_condition_list.append(problem_info_list[0])
    total_problem_memory_condition_list.append(problem_info_list[1])
    total_problem_submission_count_list.append(problem_info_list[2])
    total_problem_answer_submission_count_list.append(problem_info_list[3])
    total_problem_answered_people_count_list.append(problem_info_list[4])
    total_problem_correct_rate_list.append(problem_info_list[5])
    total_crawling_problem_number_list.append(n)

    total_problem_description_list.append(problem_description.text)

    try:
        problem_input = driver.find_element_by_css_selector('#problem_input')
        total_problem_input_list.append(problem_input.text)
    except NoSuchElementException:
        total_problem_input_list.append("정해진 입력 형식이 없습니다.")

    try:
        problem_output = driver.find_element_by_css_selector('#problem_output')
        total_problem_output_list.append(problem_output.text)
    except NoSuchElementException:
        total_problem_output_list.append("정해진 출력 형식이 없습니다.")   

    
    lists_to_save = {
        'total_problem_name_list': total_problem_name_list,
        'total_problem_tier_list': total_problem_tier_list,
        'total_problem_algorithm_list': total_problem_algorithm_list,
        'total_crawling_problem_number_list': total_crawling_problem_number_list,
        'total_problem_time_condition_list': total_problem_time_condition_list,
        'total_problem_memory_condition_list': total_problem_memory_condition_list,
        'total_problem_submission_count_list': total_problem_submission_count_list,
        'total_problem_answer_submission_count_list': total_problem_answer_submission_count_list,
        'total_problem_answered_people_count_list': total_problem_answered_people_count_list,
        'total_problem_correct_rate_list': total_problem_correct_rate_list,
        'total_problem_description_list' : total_problem_description_list,
        'total_problem_input_list' : total_problem_input_list,
        'total_problem_output_list' : total_problem_output_list,
        'remove_problem_list': remove_problem_list,
        'total_english_problem_list': total_english_problem_list
    }

    file_name = "DB_problem_table.pkl"
    with open(file_name, 'wb') as file:
        pickle.dump(lists_to_save, file) 
    

#driver.quit()

https://www.acmicpc.net/problem/1320
존재하지 않는 문제
https://www.acmicpc.net/problem/1321
https://www.acmicpc.net/problem/1322
https://www.acmicpc.net/problem/1323
https://www.acmicpc.net/problem/1324
https://www.acmicpc.net/problem/1325
https://www.acmicpc.net/problem/1326
https://www.acmicpc.net/problem/1327
https://www.acmicpc.net/problem/1328
https://www.acmicpc.net/problem/1329
https://www.acmicpc.net/problem/1330
https://www.acmicpc.net/problem/1331
https://www.acmicpc.net/problem/1332
https://www.acmicpc.net/problem/1333
https://www.acmicpc.net/problem/1334
https://www.acmicpc.net/problem/1335
https://www.acmicpc.net/problem/1336
https://www.acmicpc.net/problem/1337
https://www.acmicpc.net/problem/1338
https://www.acmicpc.net/problem/1339
https://www.acmicpc.net/problem/1340
https://www.acmicpc.net/problem/1341
https://www.acmicpc.net/problem/1342
https://www.acmicpc.net/problem/1343
https://www.acmicpc.net/problem/1344
https://www.acmicpc.net/problem/1345
https://www.acmicpc.net/pro

In [ ]:
total_problem_name_list.pop(-1)

'정ㅋ벅ㅋ'

In [ ]:
total_problem_tier_list.pop(-1)

In [65]:
print(total_problem_name_list)
print(total_problem_tier_list)
print(total_problem_algorithm_list)
print(total_crawling_problem_number_list)
print(total_problem_time_condition_list)
print(total_problem_memory_condition_list)
print(total_problem_submission_count_list)
print(total_problem_answer_submission_count_list)
print(total_problem_answered_people_count_list)
print(total_problem_correct_rate_list)
print(total_problem_description_list)
print(total_problem_input_list)
print(total_problem_output_list)
print(remove_problem_list)
print(total_english_problem_list)


['A+B', 'A-B', '터렛', '피보나치 함수', '어린 왕자', 'ACM Craft', '습격자 초라기', '벡터 매칭', 'A/B', '분산처리', '다리 놓기', 'Fly me to the Alpha Centauri', '유기농 배추', 'Contact', '컨닝', '수열 정렬', '제곱 ㄴㄴ 수', '소수 쌍', '체스판 다시 칠하기', '책 페이지', '디지털 카운터', '회전하는 큐', '소용돌이 예쁘게 출력하기', '괄호 문자열', '수열의 합', '제곱수 찾기', '보물', '고층 건물', '다이아몬드 광산', '그림 교환', '프렉탈 평면', '스타 대결', '명령 프롬프트', '칵테일', '램프', '조각 움직이기', '36진수', '약수', '감소하는 수', '교환', '정수', '주사위', '움', '거짓말', '팀 선발', '도로', '그림자', '울타리', '유니콘', '기타줄', '물약', '숫자 정사각형', '물병', '팰린드롬 공장', '팰린드롬 문장', '끝이없음', '수', '토너먼트', '친구', '좋은 구간', '좋은 수', '삼각형', '가르침', '킹', '평행사변형', '한수', '에이한수', '이동', '트리', '집으로', '김지민의 침략', '소트', '게임', '도미노', 'Z', '나누기', '저항', '넓이', '뒤집음', '마피아', '행렬', '합', '방 번호', '소트', '방 번호 2', '직사각형에서 탈출', '박성원', '쥐 잡기', '케이크', '스타트링크 타워', '체커', '카드 섞기', '배', '스티커 수집', '막대기', '마법의 구슬', '종이 접기', '마법의 문자열', '쌍둥이 마을', '알 수 없는 문장', '하얀 칸', '카드 정리 1', '발전소', '게임', '패턴', '팔', '호텔', '리모컨', '검색 엔진', '섬', '더하기 사이클', 'IQ Test', '진법 변환', '수영장 만들기', '통나무 자르기', '순열', '순열 2', '색칠 1', '색칠

In [77]:
c_n=-1
print(total_problem_name_list[c_n])
print(total_problem_tier_list[c_n])
print(total_problem_algorithm_list[c_n])
print(total_crawling_problem_number_list[c_n])
print(total_problem_time_condition_list[c_n])
print(total_problem_memory_condition_list[c_n])
print(total_problem_submission_count_list[c_n])
print(total_problem_answer_submission_count_list[c_n])
print(total_problem_answered_people_count_list[c_n])
print(total_problem_correct_rate_list[c_n])
print(total_problem_description_list[c_n])
print(total_problem_input_list[c_n])
print(total_problem_output_list[c_n])
#print(remove_problem_list[c_n])
#print(total_english_problem_list[c_n])

print(len(total_problem_name_list))
print(len(total_problem_tier_list))
print(len(total_problem_algorithm_list))
print(len(total_crawling_problem_number_list))
print(len(total_problem_time_condition_list))
print(len(total_problem_memory_condition_list))
print(len(total_problem_submission_count_list))
print(len(total_problem_answer_submission_count_list))
print(len(total_problem_answered_people_count_list))
print(len(total_problem_correct_rate_list))
print(len(total_problem_description_list))
print(len(total_problem_input_list))
print(len(total_problem_output_list))
print(len(remove_problem_list))
print(len(total_english_problem_list))

지민 vs 한수
Gold II
['브루트포스 알고리즘', '기하학', '스위핑']
1319
2
128
112
32
21
24.706
장엄지는 과일밭을 소유하고 있었다. 하지만 장엄지가 어디론가 떠나는 바람에 김지민과 임한수가 그 밭을 가지게 되었다. 과일밭은 평면으로 표현되며 과일나무는 점 하나로 표현된다.
김지민과 임한수는 과일밭을 직선을 이용해서 나누려고 한다. 단, 직선 위에는 나무가 있으면 안 된다. 이렇게 되면 김지민은 과일밭의 한 쪽을 가지게 되고, 임한수는 다른 한 쪽을 가지게 된다.
두 사람은 모두 과일 나무를 이용해서 이윤을 창출하려고 하기 때문에, 모든 나무에 각각의 가치를 매겼다. 공평하게 밭을 나누기 위해서 김지민이 가지고 있는 나무의 가치의 합과 임한수 것의 차이를 최소로 하려고 한다.
나무의 위치가 주어졌을 때, 김지민과 임한수가 가지고 있는 나무의 가치 차이의 최솟값을 구하는 프로그램을 작성하시오.
첫째 줄에 나무의 개수 N이 주어진다. N은 2보다 크거나 같고, 50보다 작거나 같은 자연수이다. 둘째 줄부터 N개의 줄에 나무의 x, y좌표와 가치가 주어진다. 좌표는 1,000보다 작거나 같은 음이 아닌 정수이고, 가치는 1,000,000보다 작거나 같은 자연수이다. 두 나무가 같은 좌표를 갖는 경우는 없다.
첫째 줄에 김지민이 가지고 있는 나무의 가치의 합과 임한수가 가지고 있는 나무의 가치의 합의 차이의 최솟값을 출력한다.
318
318
318
318
318
318
318
318
318
318
318
318
318
0
2
